In [11]:

# This is lzl's snippet for diploma project
import math
import os,shutil
import random
import warnings
import gc

import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely
import seaborn as sns
from shapely.geometry import LineString, Point, Polygon
from shapely.wkt import dumps, loads
from tqdm import tqdm

import traProject as tp
from traProject.analysisFuns import statisticFuns as sfun
from traProject.basicFuns import CalFuns as cfun,VisFuns as vfun,DataProcessingFuns as dfun
from traProject.classificationFuns import model_feature_selection as mfs
import traProject.utils as tu

random.seed(1229)

# plt.style.use('seaborn')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

warnings.filterwarnings("ignore")

picOut='../../output/pic/paper/'


In [8]:
# inpath = '../../data/wuhan/'
# data=gpd.read_file(inpath+'WUHAN.gdb',layer='打断合并后')
# data.to_csv('../../output/paper/wuhan_road/WUHAN1.csv',index=0,encoding='utf-8-sig')

In [12]:
rpath='../../output/paper/wuhan_road/WUHAN1.csv'#路网文件数据路径
rv0=pd.read_csv(rpath)
rv0['maxspeed0']=rv0['maxspeed']#maxspeed0 为原始限速字段，处理后的保留在maxspeed字段
rv0=tu.dfReC(rv0,'maxspeed0',12)
rv0

,osm_id,code,fclass,name,oneway,layer,bridge,tunnel,link_id,viaduct,numofLines,maxspeed,maxspeed0,Shape_Length,geometry
0,27317929,5112,trunk,武汉长江二桥,F,2,T,F,27317929-2,F,3,60,60,0.002124,MULTILINESTRING ((114.3053629 30.6144229999999...
1,456191653,5112,trunk,武汉长江二桥,F,2,T,F,456191653-0-456191653-1,F,3,60,60,0.002955,MULTILINESTRING ((114.3309901000001 30.5977393...
2,27318401,5112,trunk,武汉长江二桥,F,1,T,F,27318401-0,F,2,60,60,0.002363,MULTILINESTRING ((114.2997449 30.6175328000001...
3,27402443,5112,trunk,白沙洲長江大,F,1,T,F,27402443-1,F,3,"80,60","80,60",0.026617,MULTILINESTRING ((114.2505254 30.4808406999999...
4,27402545,5132,trunk_link,None,F,1,T,F,27402545-0,F,1,40,40,0.001114,MULTILINESTRING ((114.1550996 30.6165676999999...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529,881562315,5114,secondary,京汉大道 - Jing han road,F,0,F,F,881562315-8,T,3,50,50,0.000848,MULTILINESTRING ((114.2938145000002 30.5989811...
3530,882357948,5131,motorway_link,None,F,1,T,F,882357948-0,F,1,40,40,0.000492,MULTILINESTRING ((114.2927839000001 30.6681909...
3531,882357949,5131,motorway_link,None,F,1,T,F,882357949-0,F,1,40,40,0.000466,MULTILINESTRING ((114.2927839000001 30.6681909...
3532,882357955,5131,motorway_link,None,F,0,F,F,882357955-0,F,2,40,40,0.000783,MULTILINESTRING ((114.2940493 30.6708325999999...


In [13]:
# # 生成校准文件
# ms=rv0.copy()
# ms['flag']=0
# for i in range(len(ms)):
#     row=ms.iloc[i,:]
#     if len(row.maxspeed)!=2:
#         ms.loc[i,'flag']=1
# wlist=ms[ms['flag']==1]
# gs=wlist.groupby('maxspeed')
# wv=[]
# for name,g in gs:
#     wv.append(name)
# correctList=pd.DataFrame(wv,columns=['old'])
# correctList['new']=0
# correctList.to_csv('../../output/paper/wuhan_road/maxspeed1_fit.csv',encoding='utf-8-sig',index=0)#在文件中人工修正

In [14]:
clist=pd.read_csv('../../output/paper/wuhan_road/maxspeed1_fit.csv')#-1为不考虑的
clist

,old,new
0,60≤x40,-1
1,"70,40(下桥有一部分因维修为限40)",-1
2,"70,60",-1
3,"80,60",-1
4,"80,80,60",-1
5,"F30,T20",-1
6,"F40,T50",-1
7,"T30,F40",-1
8,"T40,F30",-1
9,"T50,F30",-1


In [15]:
for idx in range(len(clist)):
    co=clist.loc[idx,'old']
    cn=clist.loc[idx,'new']
    rv0.loc[rv0.maxspeed0==co,'maxspeed']=cn
rv0.maxspeed=rv0.maxspeed.astype(int)
rv0=rv0[rv0.maxspeed>0]
rv0=rv0.reset_index(drop=True)
rv0.numofLines=rv0.numofLines.apply(lambda x:int(x[0]))#处理车道数
rv0=tu.cutBZ(rv0,clist=['oneway','bridge','tunnel','viaduct'])
#viaduct 修正
rv0.loc[rv0.viaduct==' ','viaduct']='F'

In [16]:
rv0['Sdis']=rv0['geometry'].apply(lambda x:loads(x).length)#曲线长
rv0['Xdis']=rv0['geometry'].apply(tu.calXdis)#直线长
rv0['RC']=rv0['Sdis']/rv0['Xdis']#弯曲度

In [17]:
rv0.to_csv('../../output/paper/wuhan_road/WUHAN1_new.csv',index=0,encoding='utf-8-sig')